In [ ]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import calendar
import datetime as dt

import numpy as np
import fnmatch
import os

# Читаем и преобразовываем JSON

In [ ]:
print('Обработка файла JSON')

with open('sales.json', encoding='utf8') as data_file:
    json_data = json.load(data_file)
_columns=[val['Name']['#value'] for val in json_data['#value']['column']]#Вычленяем колонки
data=pd.DataFrame(data=json_data['#value']['row'], columns=_columns).applymap(lambda cell: cell['#value'])#читаем JSON и вычленяем значения

del json_data
print('Готово')

# Преобразование характеристик

In [ ]:
data['Дата']=pd.to_datetime(data['Дата'],format='%Y-%m-%d').dt.date.astype('datetime64[ns]')#преобразуем дату

ext_col=set(['ПокупательДатаРождения','ПокупательПервыйКонтакт','ПокупательДатаПереходаНаБонусную'])
for col in ext_col:
    data[col]=pd.to_datetime(data[col],format='%Y-%m-%d',errors='coerce')#преобразуем дату

data.sort_values('Дата',inplace=True)

## Финальная обработка категориальных данных

In [ ]:
#получаем колонки категориальных характеристик

categorical_columns = list(set([c for c in data.columns if data[c].dtype.name == 'object']) - set(['ПокупательСтатусАктивации','ПокупательОтказОтСМС','ПокупательПроцентСкидки']))

#заполняем пустые значения в категориальных данных
data[categorical_columns]=data[categorical_columns].applymap(lambda cell: cell.strip() if bool(cell.strip()) else '<Неопределено>')

# Сохраняем в формате Pickle (Python)

In [ ]:
#проверка на Nan, null
count=0
null_series=data.isnull().any()
for idx in null_series.index:
    if (null_series[idx])&(idx not in(ext_col)): 
        count+=1
        print (str(count)+'. '+idx) 
        

if count>0:
    print('Существуют данные null')
    raise SystemExit(2)

data.reset_index().drop(['index'], axis=1).to_pickle('MG_Sales_customer.pickle',compression='gzip')
del data

In [ ]:
print('Завершено',str(dt.datetime.now()))